Con ese notebook busco demostrar el funcionamiento completo del proyecto:
- Inferencia con modelos de Hugging Face
- Exportación a ONNX
- Benchmark en CPU
- Servidor FastAPI para inferencia REST

In [1]:
import sys
import os

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

1. Carga y prueba de pipelines

In [3]:
from inference.pipelines import generate_text, answer_question

c:\Users\calag\Desktop\nlp-proyecto4\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
Device set to use cpu


In [5]:
#PRUEBA 1: GENERACIÓN DE TEXTO
prompt = "Once upon a time in a galaxy"
generated = generate_text(prompt, max_length=40)

print("Texto generado:\n", generated)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=40) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Texto generado:
 Once upon a time in a galaxy where the galaxy is dark, the galaxy has a bright, bright galaxy with a dark red galaxy. When it is dark, the galaxy has a dark blue galaxy. The galaxy has a bright red galaxy. A galaxy as bright as this galaxy, the galaxy has a dark red galaxy. When it is dark, the galaxy has a bright red galaxy. The galaxy has a bright red galaxy. When it is dark, the galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. When it is dark, the galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galaxy has a bright red galaxy. The galax

In [6]:
#PRUEBA 2: PREGUNTA Y RESPUESTA
context = "La UNI está ubicada en Lima, Perú y fue fundada en 1876"
question = "¿Cuándo fue fundada la UNI?"
answer = answer_question(context, question)

print("Respuesta:", answer)

Respuesta: 1876


2. Prueba del modelo ONNX cargado

In [7]:
import onnxruntime as ort
from transformers import AutoTokenizer

In [9]:
model_path = "../export/model_qa.onnx"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")

context = "La UNI fue fundada en 1876 y está en Lima."
question = "¿Dónde está la UNI?"

inputs = tokenizer(question, context, return_tensors="np")
session = ort.InferenceSession(model_path)
outputs = session.run(None, {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
})

In [10]:
start = outputs[0].argmax()
end = outputs[1].argmax() + 1
respuesta = tokenizer.decode(inputs["input_ids"][0][start:end])
print("Respuesta con ONNX:", respuesta)

Respuesta con ONNX: Lima


3. Comparación del tiempo de inferencia (benchmark)

In [12]:
!python ../benchmarks/benchmark.py


Resultados de Benchmark:

 modelo  batch_size  latencia_s  throughput
pytorch          16    0.011466   87.214400
   onnx          16    0.008502  117.616407


4. Prueba con batch y padding dinámico

In [13]:
from inference.batching import *

 ¿Dónde está la UNI?
 Respuesta: Lima

 ¿Cuándo fue fundada la UNI?
 Respuesta: La Universidad Nacional de Ingeniería fue fundada en 1876 y está ubicada en Lima, Perú

 ¿Cómo se conoce también a la UNI?
 Respuesta: conocida como Universidad Nacional de Ingeniería



5. Prueba del servidor FastAPI (si ya está corriendo)

http://localhost:8000/docs